In [1]:
%%configure -f
{
    "executorMemory": "5G",
    "executorCores" : 4
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1614566963109_0004,pyspark,idle,Link,Link,
7,application_1614566963109_0008,pyspark,idle,Link,Link,


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import Row
# from pyspark.sql.function import *
from pyspark.sql.functions import isnan, isnull

ss = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1614566963109_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = ss.read.csv('s3://msds694-dataset/CA_2013_onwards.csv', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 1. Check all the column names

In [4]:
column_names = df.columns
column_names

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['id', 'state', 'stop_date', 'stop_time', 'location_raw', 'county_name', 'county_fips', 'fine_grained_location', 'police_department', 'driver_gender', 'driver_age_raw', 'driver_age', 'driver_race_raw', 'driver_race', 'violation_raw', 'violation', 'search_conducted', 'search_type_raw', 'search_type', 'contraband_found', 'stop_outcome', 'is_arrested', 'ethnicity']

## 2. Check if there exsits empty columns

In [5]:
count_rows = df.count()
count_rows

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14536338

In [6]:
for col in column_names:
    count_nan = df.filter(isnull(col)).count()
    print(col + ' has ' + str(count_nan) + ' empty rows ')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

id has 0 empty rows 
state has 0 empty rows 
stop_date has 0 empty rows 
stop_time has 14536338 empty rows 
location_raw has 4 empty rows 
county_name has 1076319 empty rows 
county_fips has 1076319 empty rows 
fine_grained_location has 14536338 empty rows 
police_department has 14536338 empty rows 
driver_gender has 0 empty rows 
driver_age_raw has 0 empty rows 
driver_age has 14536338 empty rows 
driver_race_raw has 0 empty rows 
driver_race has 0 empty rows 
violation_raw has 0 empty rows 
violation has 0 empty rows 
search_conducted has 0 empty rows 
search_type_raw has 0 empty rows 
search_type has 14057611 empty rows 
contraband_found has 456292 empty rows 
stop_outcome has 0 empty rows 
is_arrested has 0 empty rows 
ethnicity has 0 empty rows

### There exsist some empty columns: ['stop_time', 'fine_grained_location', 'police_department', 'driver_age', 'search_type'], so we will drop them

## 3. Check if there exists meaningless columns

In [7]:
df.select('id').show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+
|             id|
+---------------+
|CA-2013-0000001|
|CA-2013-0000002|
|CA-2013-0000003|
|CA-2013-0000004|
|CA-2013-0000005|
+---------------+
only showing top 5 rows

### id means violation id, which is not related to target so we will drop it otherwise the final model could overfit

In [8]:
df.select('state').groupBy('state').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+
|state|   count|
+-----+--------+
|   CA|14536338|
+-----+--------+

### This dataset is only about CA so this column is useless, drop it

## 4. Check if there exsist paired columns that have same meaning

In [9]:
df.select('county_name').groupBy('county_name').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|         county_name|  count|
+--------------------+-------+
|       Sutter County|  84115|
|       Colusa County|  47871|
|       Madera County|  84561|
|         Yolo County|  76907|
|    Del Norte County|  35058|
|       Solano County| 262531|
|    Riverside County| 532606|
|        Modoc County|  20459|
|   Sacramento County| 505118|
|       Merced County| 198098|
|     Humboldt County|  63186|
|   Stanislaus County| 135572|
|San Bernardino Co...| 867974|
|         Mono County|  48007|
|                null|1076319|
|       Fresno County| 269988|
|  Santa Clara County| 299475|
|        Kings County|  74262|
|       Tehama County| 103669|
|       Alpine County|  43413|
+--------------------+-------+
only showing top 20 rows

In [10]:
df.select('county_fips').groupBy('county_fips').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------+
|county_fips|  count|
+-----------+-------+
|      06063|  30284|
|      06079| 147460|
|      06057|  94883|
|      06013| 173356|
|      06065| 532606|
|      06045| 126502|
|      06005|  38521|
|      06109|  54473|
|      06085| 299475|
|      06099| 135572|
|      06031|  74262|
|       null|1076319|
|      06069| 197610|
|      06035|  56636|
|      06017|  78010|
|      06077| 254279|
|      06019| 269988|
|      06033|  38911|
|      06001| 834219|
|      06081| 217778|
+-----------+-------+
only showing top 20 rows

### 'county_name' and 'county_fips' have same meaning, we'll choose 'county_fips' for convenience and drop the other one

In [11]:
df.select('driver_race_raw').groupBy('driver_race_raw').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|     driver_race_raw|  count|
+--------------------+-------+
|        Asian Indian|  28063|
|              Indian|  37413|
|Other Pacific Isl...|   7585|
|             Chinese|   8845|
|            Hawaiian|    612|
|            Japanese|   1931|
|            Filipino|  17955|
|          Vietnamese|   5226|
|               Other|1126556|
|              Samoan|   1528|
|           Guamanian|    673|
|              Korean|   3829|
|             Laotian|    282|
|               White|6184252|
|           Cambodian|    382|
|            Hispanic|4954207|
|               Black|1221604|
|         Other Asian| 935395|
+--------------------+-------+

In [12]:
df.select('driver_race_raw').groupBy('driver_race_raw').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|     driver_race_raw|  count|
+--------------------+-------+
|        Asian Indian|  28063|
|              Indian|  37413|
|Other Pacific Isl...|   7585|
|             Chinese|   8845|
|            Hawaiian|    612|
|            Japanese|   1931|
|            Filipino|  17955|
|          Vietnamese|   5226|
|               Other|1126556|
|              Samoan|   1528|
|           Guamanian|    673|
|              Korean|   3829|
|             Laotian|    282|
|               White|6184252|
|           Cambodian|    382|
|            Hispanic|4954207|
|               Black|1221604|
|         Other Asian| 935395|
+--------------------+-------+

In [13]:
df.select('ethnicity').groupBy('ethnicity').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+-------+
|ethnicity|  count|
+---------+-------+
|        K|   3829|
|        F|  17955|
|        B|1221604|
|        L|    282|
|        V|   5226|
|        U|    612|
|        O|1126556|
|        D|    382|
|        C|   8845|
|        J|   1931|
|        Z|  28063|
|        A| 935395|
|        W|6184252|
|        S|   1528|
|        G|    673|
|        I|  37413|
|        P|   7585|
|        H|4954207|
+---------+-------+

### 'driver_race_raw', 'ethnicity' are similar, we'll choose 'driver_race' and drop the other one

## 5. Make a new df 

In [14]:
newdf = df.drop('id','state','stop_time','county_name', 'fine_grained_location', 'police_department', 'driver_age', 'stop_outcome', 'ethnicity')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
newdf.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['stop_date', 'location_raw', 'county_fips', 'driver_gender', 'driver_age_raw', 'driver_race_raw', 'driver_race', 'violation_raw', 'violation', 'search_conducted', 'search_type_raw', 'search_type', 'contraband_found', 'is_arrested']

## 6.Impute Missing Values 

- location_raw

In [16]:
# number of missing value in column location_raw
newdf.filter(newdf['location_raw'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4

In [17]:
# get the most common value of location_raw and replace null value with it.
mode = newdf.groupBy(newdf['location_raw']).count().orderBy('count', ascending=False).first()[0]
newdf = newdf.fillna(mode, subset='location_raw')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
newdf.filter(newdf['location_raw'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

- county_fips

In [19]:
# number of missing value in column county_fips
newdf.filter(newdf['county_fips'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1076319

In [20]:
location_county = newdf.filter(newdf['county_fips'].isNull()).select('location_raw','county_fips')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
location_county.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|        location_raw|county_fips|
+--------------------+-----------+
|          Mt. Shasta|       null|
|          Mt. Shasta|       null|
|          Mt. Shasta|       null|
|          Mt. Shasta|       null|
|          Mt. Shasta|       null|
|          Mt. Shasta|       null|
|          Mt. Shasta|       null|
|Conejo Inspection...|       null|
|Conejo Inspection...|       null|
|Conejo Inspection...|       null|
|Conejo Inspection...|       null|
|Conejo Inspection...|       null|
|Conejo Inspection...|       null|
|Conejo Inspection...|       null|
|            Moorpark|       null|
|            Moorpark|       null|
|            Moorpark|       null|
|            Moorpark|       null|
|            Moorpark|       null|
|            Temecula|       null|
+--------------------+-----------+
only showing top 20 rows

In [22]:
location_county.groupBy(newdf['location_raw']).count().orderBy('count', ascending=False).show()
# we can observe that many locations have corresponding county_fips as null, however, replacing these null values 
# with the most common values doens't make any sense.
# And even we replace null county_fips with their corresponding values, still there are several locations represented 
# with a number, which doesn't contain information to search it's corresponding county. 
# Conclusion: we can just drop 'county_fips' column.

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
|        location_raw| count|
+--------------------+------+
|             Ventura|157996|
|            Moorpark|141818|
|            Temecula|131368|
|       Santa Barbara|106447|
|            Coalinga| 99385|
|             Buelton| 72130|
|         Santa Maria| 62077|
|               Yreka| 52646|
|Dunsmuir Grade In...| 49006|
|Rainbow Inspectio...| 48488|
|Conejo Inspection...| 45275|
|                  25| 39678|
|          Mt. Shasta| 35439|
|     San Luis Obispo| 30849|
|                  62|  2321|
|                  61|   728|
|                  91|   434|
|                   3|    69|
|                  33|    30|
|                  29|    29|
+--------------------+------+
only showing top 20 rows

In [23]:
newdf = newdf.drop('county_fips')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
# number of missing value in column county_fips
newdf.filter(newdf['search_type'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14057611

In [25]:
# We observe most search_type are not recorded. In order to not lose too much information, combining null values and
# type 'Other' seems to be a good idea. 
newdf.groupBy(newdf['search_type']).count().orderBy('count', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------+
|       search_type|   count|
+------------------+--------+
|              null|14057611|
|Incident to Arrest|  275590|
|         Inventory|  145622|
|  Protective Frisk|   27745|
|    Probable Cause|   18169|
|           Warrant|    6232|
|           Consent|    4266|
|             Other|    1103|
+------------------+--------+

In [26]:
# replace null values with 'Other'
newdf = newdf.fillna('Other', subset='search_type')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- contraband_found

In [27]:
# number of missing value in column contraband_found
newdf.filter(newdf['contraband_found'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

456292

In [28]:
# replace null values with the most common value
mode = newdf.groupBy(newdf['contraband_found']).count().orderBy('count', ascending=False).first()[0]
newdf = newdf.fillna(mode, subset='contraband_found')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
newdf.filter(newdf['contraband_found'].isNull()).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

## 7.Data Preprocessing - StringIndexer & OneHotEncoder

In [30]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import VectorAssembler

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
categorical_col = newdf.columns[1:]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
def Col_Str(df, cols):
    newdf = df

    for c in cols:
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf


def OHE(df, cols):
    newdf = df
    for c in cols:
        ohe = OneHotEncoder(inputCol=c, outputCol=c+"-onehot", dropLast=False)
#         ohe_model = ohe.fit(newdf)

        newdf = ohe.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-onehot", c)
    return newdf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
processed_df = Col_Str(newdf, categorical_col)

processed_df = OHE(processed_df, categorical_col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
processed_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+-------------+--------------+---------------+-------------+-------------+-------------+----------------+---------------+-------------+----------------+-------------+
| stop_date|   location_raw|driver_gender|driver_age_raw|driver_race_raw|  driver_race|violation_raw|    violation|search_conducted|search_type_raw|  search_type|contraband_found|  is_arrested|
+----------+---------------+-------------+--------------+---------------+-------------+-------------+-------------+----------------+---------------+-------------+----------------+-------------+
|2013-01-01|(145,[0],[1.0])|(2,[0],[1.0])| (6,[0],[1.0])| (18,[1],[1.0])|(5,[1],[1.0])|(8,[1],[1.0])|(4,[2],[1.0])|   (2,[0],[1.0])| (10,[0],[1.0])|(7,[0],[1.0])|   (2,[0],[1.0])|(2,[0],[1.0])|
|2013-01-01|(145,[0],[1.0])|(2,[1],[1.0])| (6,[2],[1.0])| (18,[2],[1.0])|(5,[2],[1.0])|(8,[0],[1.0])|(4,[0],[1.0])|   (2,[0],[1.0])| (10,[0],[1.0])|(7,[0],[1.0])|   (2,[0],[1.0])|(2,[0],[1.0])|
|2013-01-01|(145,[0],[1.0])|(2

## 8.save back to s3 

In [42]:
processed_df.write.parquet('s3a://msds694-dataset/CA_2013_onwards_new.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 9. Check saved files 

In [44]:
check_df = ss.read.parquet('s3a://msds694-dataset/CA_2013_onwards_new.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
check_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------------+-------------+--------------+---------------+-------------+-------------+-------------+----------------+---------------+-------------+----------------+-------------+
| stop_date|    location_raw|driver_gender|driver_age_raw|driver_race_raw|  driver_race|violation_raw|    violation|search_conducted|search_type_raw|  search_type|contraband_found|  is_arrested|
+----------+----------------+-------------+--------------+---------------+-------------+-------------+-------------+----------------+---------------+-------------+----------------+-------------+
|2013-07-10|(145,[10],[1.0])|(2,[0],[1.0])| (6,[4],[1.0])| (18,[4],[1.0])|(5,[4],[1.0])|(8,[1],[1.0])|(4,[2],[1.0])|   (2,[0],[1.0])| (10,[0],[1.0])|(7,[0],[1.0])|   (2,[0],[1.0])|(2,[0],[1.0])|
|2013-07-10|(145,[10],[1.0])|(2,[0],[1.0])| (6,[0],[1.0])| (18,[0],[1.0])|(5,[0],[1.0])|(8,[2],[1.0])|(4,[1],[1.0])|   (2,[0],[1.0])| (10,[0],[1.0])|(7,[0],[1.0])|   (2,[0],[1.0])|(2,[0],[1.0])|
|2013-07-10|(145,[10],[1.